In [16]:
import sqlite3
import json
from datetime import datetime

timeframe = '2018-02'
sql_transaction = []

connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()


In [17]:
def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply(
    parent_id TEXT PRIMARY KEY,
    comment_id TEXT UNIQUE,
    parent TEXT,
    comment TEXT,
    subreddit TEXT,
    unix INT,
    score INT)""")
    

def format_data(data):
    data = data.replace("\n"," newlinechar ").replace("\r"," newlinechar ").replace('"',"'")
    return data    

def find_parent(pid):
    """
    - get body of the parent comment
    """
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid);
        c.execute(sql)
        result = c.fetchone()
        if result!= None:
            return result[0]
        else: return False
    except Exception as e:
        #print("")
        return False
    
def find_existing_score(pid):
    """
    - find the score of the comment which has same parent_id
    """
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid);
        c.execute(sql)
        result = c.fetchone()
        if result!= None:
            return result[0]
        else: return False
    except Exception as e:
        #print("")
        return False

In [18]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) <1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data== '[removed]':
        return False
    else:
        return True

def sql_insert_replace_comment(commentid, parentid, parent, comment, subreddit, time,score):
    """ parent : body of the parent comment
        comment : body of current comment
        - replace the current child comment with current comment based on parent_id
    """
    try:
        sql = """ UPDATE parent_reply SET parent_id = ?,comment_id = ?,parent=?, comment= ?, subreddit = ?, unix = ?, score = ? WHERE parent_id = ? ;  """.format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)    
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))
        
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    """ parent : body of the parent
        comment : body of the current comment'
        - insert the child comment of any parent
    """
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))


def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    """ comment: body
        time : create_utc
        - insert the data as no parent is there
    """
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))
        
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []
        

In [14]:
import json
create_table()

In [15]:
row_counter = 0

with open("./data/RC_2018-02-28",buffering=10) as f:
    for row in f:
        row = json.loads(row)
        row_counter += 1
        parent_id = row['parent_id']
        body = format_data(row['body'])
        created_utc = row['created_utc']
        score = row['score']
        comment_id = row['link_id']
        subreddit = row['subreddit']       
        parent_data = find_parent(parent_id)
        # get body of parent
        #print(parent_data)
        
        # if comment score > 2 then only we will consider adding the comment
        if score >=2:
            # get existing score of comment having same parent_id, may or may not have a parent            
            existing_comment_score = find_existing_score(parent_id)
            
            # if there is a comment with same parent_id
            if existing_comment_score:
                
                # if new score is greater, replace the old and add body of parent also to it
                if score > existing_comment_score:
                    if acceptable(body): 
                        sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            
            else:
                # if there is no comment with same parent_id
                if acceptable(body):
                    if parent_data:
                        # if has parent, add body of parent as parent_data and insert
                        sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                    else:
                        # if does not have a parent, directly insert the comment
                        sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)

        if row_counter % 100000 == 0:
            print("Total rows read :{}, Time: {}".format(row_counter, str(datetime.now())))

Total rows read :100000, Time: 2018-10-30 19:32:25.615695
Total rows read :200000, Time: 2018-10-30 19:32:45.657143
Total rows read :300000, Time: 2018-10-30 19:33:05.526402
Total rows read :400000, Time: 2018-10-30 19:33:27.947107
Total rows read :500000, Time: 2018-10-30 19:33:50.338591
Total rows read :600000, Time: 2018-10-30 19:34:10.654832
Total rows read :700000, Time: 2018-10-30 19:34:29.593693
Total rows read :800000, Time: 2018-10-30 19:34:52.047817
Total rows read :900000, Time: 2018-10-30 19:35:11.908802
Total rows read :1000000, Time: 2018-10-30 19:35:32.526707
Total rows read :1100000, Time: 2018-10-30 19:35:51.252181
Total rows read :1200000, Time: 2018-10-30 19:36:10.275219
Total rows read :1300000, Time: 2018-10-30 19:36:28.988859
Total rows read :1400000, Time: 2018-10-30 19:36:48.049140
Total rows read :1500000, Time: 2018-10-30 19:37:07.109592
Total rows read :1600000, Time: 2018-10-30 19:37:26.023422
Total rows read :1700000, Time: 2018-10-30 19:37:46.095767
Total 

KeyboardInterrupt: 